In [61]:
import cv2
import pandas
import numpy as np
from tqdm import tqdm
from typing import Tuple

PARTICIPANT_DATA = "../data/eye_tracking/raw/experience1/session1/nov02/1451_10.csv"
OUTPUT_FILE_PATH = "../data/result.mp4"
FRAME_WIDTH = 6000
FRAME_HEIGHT = 3000
DOWNSCALE_FACTOR = 4
FPS = 120
CIRCLE_RADIUS = 10
CIRCLE_COLOR = (0, 0, 255)

In [62]:
session_df = pandas.read_csv(PARTICIPANT_DATA, sep=";")
sequence_df = session_df.groupby("SequenceId")

def preprocess_session_df(
    session_df: pandas.DataFrame,
    frame_width: int,
    frame_height: int,
) -> pandas.DataFrame:
    session_df = session_df.copy()
    session_df = session_df[
        (session_df["GazeX"] != int(frame_width / 2)) & (session_df["GazeY"] != (frame_height / 2))
    ]

    return session_df

In [63]:
session_df.head()

,Timestamp,Id,SequenceId,SequenceSet,VectorGazeX,VectorGazeY,VectorGazeZ,GazeX,GazeY
0,638345125377378054,1451.0,15.0,1.0,0.000000,0.000000,0,3000.000000,1500.000000
1,638345125377860736,1451.0,15.0,1.0,-0.055716,-0.284484,0.9570603,2791.065711,2922.419995
2,638345125378344678,1451.0,15.0,1.0,-0.059726,-0.310998,0.948532,2776.027012,3000.000000
3,638345125379243250,1451.0,15.0,1.0,-0.079245,-0.369785,0.9257318,2702.829458,3000.000000
4,638345125379725391,1451.0,15.0,1.0,-0.068195,-0.259631,0.9632971,2744.269112,2798.153996


In [ ]:
def preprocess_session(session_df: pandas.DataFrame) -> pandas.DataFrame:
    session_df = session_df.sort_values("Timestamp")
    session_df["Timestamp"] = session_df["Timestamp"] - session_df["Timestamp"].iloc[0]
    return session_df

In [68]:
# Take first sequenceid
sequence_df = df.groupby("SequenceId")
first_sequence_id = list(sequence_df.groups.keys())[0]
first_sequence_df = sequence_df.get_group(first_sequence_id)

# Remove the entries with gaze exactly in the middle, as they are not valid
first_sequence_df = first_sequence_df[
    (first_sequence_df["GazeX"] != int(FRAME_WIDTH / 2)) & (first_sequence_df["GazeY"] != (FRAME_HEIGHT / 2))
]
first_sequence_df.head()

,Timestamp,Id,SequenceId,SequenceSet,VectorGazeX,VectorGazeY,VectorGazeZ,GazeX,GazeY
4368,638345128472482010,1451.0,1.0,1.0,0.004388,-0.083184,0.9965246,3016.456639,1915.921248
4369,638345128472925470,1451.0,1.0,1.0,0.008416,-0.098157,0.9951354,3031.559626,1990.785018
4370,638345128473682579,1451.0,1.0,1.0,0.020475,-0.073645,0.9970742,3076.782410,1868.227437
4371,638345128474149677,1451.0,1.0,1.0,0.035479,-0.066680,0.9971435,3133.045460,1833.399065
4372,638345128474645850,1451.0,1.0,1.0,0.039928,-0.065270,0.9970685,3149.729247,1826.350704


In [69]:
def visualize_gaze(
    sequence_df: pandas.DataFrame,
    output_file_path: str,
    frame_width: int,
    frame_height: int,
    downscale_factor: float,
    fps: int,
    circle_radius: int,
    circle_color: Tuple[int, int, int],
):
    sequence_df = sequence_df.copy()
    
    # Scale frame resolution and gaze coordinates
    scaled_frame_width = int(frame_width / downscale_factor)
    scaled_frame_height = int(frame_height / downscale_factor)
    sequence_df['GazeX'] = sequence_df['GazeX'] / downscale_factor
    sequence_df['GazeY'] = sequence_df['GazeY'] / downscale_factor

    # Compute time difference between consecutive frames
    sequence_df = sequence_df.sort_values("Timestamp")
    sequence_df['TimeDiff'] = sequence_df['Timestamp'].diff().fillna(0)
    sequence_df['FrameDiff'] = sequence_df['TimeDiff'] / 1e7 * FPS # 100-nanosecond interval since January 1, 1601 (UTC)
    sequence_df['FrameNumber'] = sequence_df['FrameDiff'].cumsum().astype(int)

    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc('H','2','6','4')
    out = cv2.VideoWriter(output_file_path, fourcc, fps, (scaled_frame_width, scaled_frame_height))

    background = np.zeros((scaled_frame_height, scaled_frame_width, 3), dtype=np.uint8)

    frame_id = 0
    next_frame_id = sequence_df['FrameNumber'].iloc[0]
    max_frame_id = sequence_df['FrameNumber'].max()
    bar = tqdm(total=max_frame_id, desc="Generating gaze video...", unit="frames")
    while frame_id < max_frame_id:
        if frame_id == next_frame_id:
            frame = background.copy()

            x = int(sequence_df[sequence_df['FrameNumber'] == frame_id]['GazeX'].iloc[0])
            y = int(sequence_df[sequence_df['FrameNumber'] == frame_id]['GazeY'].iloc[0])

            scaled_circle_radius = int(circle_radius / downscale_factor)
            cv2.circle(frame, (x, y), radius=scaled_circle_radius, color=circle_color, thickness=-1)

            next_frame_id = sequence_df[sequence_df['FrameNumber'] > frame_id]['FrameNumber'].iloc[0]

        out.write(frame)
        frame_id += 1
        bar.update(1)

    out.release()
    cv2.destroyAllWindows()

visualize_gaze(
    first_sequence_df, 
    OUTPUT_FILE_PATH, 
    FRAME_WIDTH, 
    FRAME_HEIGHT, 
    DOWNSCALE_FACTOR,
    FPS, 
    CIRCLE_RADIUS, 
    CIRCLE_COLOR
)

Generating gaze video...: 100%|██████████| 11978/11978 [00:31<00:00, 379.96frames/s]
